# GBM(Gradient Boosting Machine)

### GBM의 개요 및 실습

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import time
import matplotlib.pyplot as plt
%matplotlib inline
import graphviz
import warnings
warnings.filterwarnings('ignore')

In [2]:
#원본 피처명에 _1 또는 _2를 추가로 부여해 새로운 피처명을 가지는 DataFrame을 반환
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data = old_feature_name_df.groupby('column_name').cumcount(), columns = ['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how = 'outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0] + '_' + str(x[1]) if x[1] > 0 else x[0], axis = 1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis = 1)
    return new_feature_name_df

In [3]:
def get_human_dataset():
    
    feature_name_df = pd.read_csv('C:/Users/creade/data_science/파이썬 머신러닝 완벽 가이드/human_activity/human_activity/features.txt'
                                 , sep = '\s+', header = None, names = ['column_index', 'column_name'])
    
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    X_train = pd.read_csv('C:/Users/creade/data_science/파이썬 머신러닝 완벽 가이드/human_activity/human_activity/train/X_train.txt'
                         , sep = "\s+", names = feature_name)
    X_test = pd.read_csv('C:/Users/creade/data_science/파이썬 머신러닝 완벽 가이드/human_activity/human_activity/test/X_test.txt'
                         , sep = "\s+", names = feature_name)
    y_train = pd.read_csv('C:/Users/creade/data_science/파이썬 머신러닝 완벽 가이드/human_activity/human_activity/train/y_train.txt'
                         , sep = "\s+", header = None, names = ['action'])
    y_test = pd.read_csv('C:/Users/creade/data_science/파이썬 머신러닝 완벽 가이드/human_activity/human_activity/test/y_test.txt'
                         , sep = "\s+", header = None, names = ['action'])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))
print('GBM 정확도 수행 시간: {0:.4f}'.format(time.time() - start_time))

In [ ]:
params = {'n_estimators' : [100, 500], 'learning_rate' : [0.05, 0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid = params, cv = 2, verbose = 1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터 :', grid_cv.best_params_)
print('최고 예측 정확도 : {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))